In [1]:
from utils import accessors
#import pandas as pd

In [2]:
show_outputs = True

In [3]:
path = "dataset/"
add_file = "addresses.jsonl"
cit_file = "cities.jsonl"

add_df = accessors.load_json(path, add_file) 
cit_df = accessors.load_json(path, cit_file) 

if show_outputs: 
    display(add_df)
    display(cit_df)

,address,country
0,Mitterweg Angath,AT
1,"josef-franz-huter-straße 72 6020, innsbruck, a...",AT
2,"FLIRSCHBERG FLIRSCH, ÖSTERREICH",AT
3,Lochbödele Landeck Austria,AT
4,"Ainet 75 9951, Ainet",AT
...,...,...
595491,RUE DU JEU DE PAUME VILLEPREUX,FR
595492,"78955, CARRIÈRES-SOUS-POISSY",FR
595493,rue du cèdre - morainvilliers,FR
595494,"Boissets, France",FR


,city,country
0,Kappl,AT
1,Schmirn,AT
2,Zams,AT
3,Wattenberg,AT
4,Polling in Tirol,AT
...,...,...
77476,Saint-Mesmin,FR
77477,Vernoux-en-Vivarais,FR
77478,Sedze-Maubecq,FR
77479,Bulat-Pestivien,FR


In [5]:
ft = add_file.split('.')[-1]
print(ft)

jsonl
